In [ ]:
import pandas as pd
import numpy as np
import common

from pyecharts.charts import Grid, Kline, Line, Bar
from pyecharts import options as opts
from pyecharts.globals import ThemeType

import talib.abstract as ta

获取数据

In [ ]:
file = '../freqtrade/user_data/data/binance/futures/ETH_USDT_USDT-8h-futures.json'

In [ ]:
df = common.read_kline_data(file)

添加MA和EMA

In [ ]:
df['ma20'] = ta.MA(df, timeperiod=20)
df['ma60'] = ta.MA(df, timeperiod=60)
df['ma120'] = ta.MA(df, timeperiod=120)

df['ema20'] = ta.EMA(df, timeperiod=20)
df['ema60'] = ta.EMA(df, timeperiod=60)
df['ema120'] = ta.EMA(df, timeperiod=120)

计算密集程度

In [ ]:
df['intensity_ma'] = (abs(df['ma20'] - df['ma60']) + abs(df['ma60'] - df['ma120']) + abs(df['ma120'] - df['ma20'])) /\
                    ((df['ma20'] + df['ma60'] + df['ma120']) / 3)
df.tail()

In [ ]:
df = df.dropna()

### 构建图表

初始化K线数据

In [ ]:
data = []
for _, row in df.iterrows():
    data.append([
        row["open"], 
        row["close"], 
        row["low"], 
        row["high"], 
    ])

开始构建K线

In [ ]:
kline = (
    Kline(init_opts=opts.InitOpts(theme=ThemeType.CHALK))
    .add_xaxis(df["datetime"].tolist())
    .add_yaxis("K线图", data)
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(is_scale=True),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            # 对数坐标
            type_="log",
            min_="dataMin",
            max_="dataMax",
            ),
        title_opts=opts.TitleOpts(title="K线图"),

        datazoom_opts=[
            opts.DataZoomOpts(
                is_show=True,
                xaxis_index=[0, 1],
                type_="inside",
                pos_top="85%",
                range_start=98,
                range_end=100,
            ),
        ],
        # 十字线坐标
        tooltip_opts=opts.TooltipOpts(
            trigger="axis",
            axis_pointer_type="cross",
            # background_color="rgba(245, 245, 245, 0.8)",
            border_width=1,
            border_color="#ccc",
            textstyle_opts=opts.TextStyleOpts(color="#000"),
        ),
        # 上下十字线触发
        axispointer_opts=opts.AxisPointerOpts(
            is_show=True,
            link=[{"xAxisIndex": "all"}],
            label=opts.LabelOpts(background_color="#777"),
        ),      
    )
)

构建K线的指标MA和EMA

In [ ]:
line = (
    Line()
    .add_xaxis(xaxis_data=df["datetime"].tolist())
    .add_yaxis(
        series_name="MA20",
        y_axis=df['ma20'],
        is_smooth=True,
        is_hover_animation=False,
        linestyle_opts=opts.LineStyleOpts(width=1, opacity=0.8),
        label_opts=opts.LabelOpts(is_show=False),
        is_symbol_show=False,
    )
    .add_yaxis(
        series_name="MA60",
        y_axis=df['ma60'],
        is_smooth=True,
        is_hover_animation=False,
        linestyle_opts=opts.LineStyleOpts(width=1, opacity=0.8),
        label_opts=opts.LabelOpts(is_show=False),
        is_symbol_show=False,
    )
    .add_yaxis(
        series_name="MA120",
        y_axis=df['ma120'],
        is_smooth=True,
        is_hover_animation=False,
        linestyle_opts=opts.LineStyleOpts(width=1, opacity=0.8),
        label_opts=opts.LabelOpts(is_show=False),
        is_symbol_show=False,
    )
    .add_yaxis(
        series_name="EMA20",
        y_axis=df['ema20'],
        is_smooth=True,
        is_hover_animation=False,
        linestyle_opts=opts.LineStyleOpts(width=1, opacity=0.8),
        label_opts=opts.LabelOpts(is_show=False),
        is_symbol_show=False,
    )
    .add_yaxis(
        series_name="EMA60",
        y_axis=df['ema60'],
        is_smooth=True,
        is_hover_animation=False,
        linestyle_opts=opts.LineStyleOpts(width=1, opacity=0.8),
        label_opts=opts.LabelOpts(is_show=False),
        is_symbol_show=False,
    )
    .add_yaxis(
        series_name="EMA120",
        y_axis=df['ema120'],
        is_smooth=True,
        is_hover_animation=False,
        linestyle_opts=opts.LineStyleOpts(width=1, opacity=0.8),
        label_opts=opts.LabelOpts(is_show=False),
        is_symbol_show=False,
    )
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(
            type_="category",
            is_scale=True,
            grid_index=1,
            boundary_gap=False,
            axisline_opts=opts.AxisLineOpts(is_on_zero=False),
            axistick_opts=opts.AxisTickOpts(is_show=False),
            splitline_opts=opts.SplitLineOpts(is_show=False),
            axislabel_opts=opts.LabelOpts(is_show=False),
            split_number=20,
            min_="dataMin",
            max_="dataMax",
        ),
        yaxis_opts=opts.AxisOpts(
            grid_index=1,
            is_scale=True,
            split_number=2,
            # axislabel_opts=opts.LabelOpts(is_show=False),
            # axisline_opts=opts.AxisLineOpts(is_show=False),
            # axistick_opts=opts.AxisTickOpts(is_show=False),
            # splitline_opts=opts.SplitLineOpts(is_show=False),
        ),
        legend_opts=opts.LegendOpts(is_show=False),
    )
)

成交量

In [ ]:
bar = (
    Bar()
    .add_xaxis(xaxis_data=df["datetime"].tolist())
    .add_yaxis(
        series_name="volume",
        y_axis=df["volume"].tolist(),
        xaxis_index=1,
        yaxis_index=1,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(
            type_="category",
            is_scale=True,
            grid_index=1,
            boundary_gap=False,
            axisline_opts=opts.AxisLineOpts(is_on_zero=False),
            axistick_opts=opts.AxisTickOpts(is_show=False),
            splitline_opts=opts.SplitLineOpts(is_show=False),
            axislabel_opts=opts.LabelOpts(is_show=False),
            split_number=20,
            min_="dataMin",
            max_="dataMax",
        ),
        yaxis_opts=opts.AxisOpts(
            grid_index=1,
            is_scale=True,
            split_number=2,
            axislabel_opts=opts.LabelOpts(is_show=False),
            axisline_opts=opts.AxisLineOpts(is_show=False),
            axistick_opts=opts.AxisTickOpts(is_show=False),
            splitline_opts=opts.SplitLineOpts(is_show=False),
        ),
        legend_opts=opts.LegendOpts(is_show=False),
    )
)

合并

In [ ]:
# Kline And Line
overlap_kline_line = kline.overlap(line)

In [ ]:
grid_chart = Grid(init_opts=opts.InitOpts(width='100%', height='600px'))

In [ ]:
grid_chart.add(
    overlap_kline_line,
    grid_opts=opts.GridOpts(pos_left="2%", pos_right="5%", height="60%"),
)

grid_chart.add(
    bar,
    grid_opts=opts.GridOpts(
        pos_left="2%", pos_right="2%", pos_top="78%", height="16%"
    ),
)

In [ ]:
grid_chart.render_notebook()

In [ ]:
s_r = df[df['intensity_ma'] < 0.02]

In [ ]:
s_r